In [3]:
import inspect
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import importlib.util

#from smarty.neuralNetworks import callbacks
#from smarty.neuralNetworks.configs import AdamConfig, ExponentialDecayConfig, FlatLrConfig, LbfgsConfig, SGDConfig
#from smarty import TORCH_AVAILABLE
#from smarty import doe

import sys
import os

#from smarty.neuralNetworks.neuralNetworksMisc import CreateTensorDataset
#from smarty.neuralNetworks.trainer import DatasetTrainer


#if TORCH_AVAILABLE:
import torch
from torch.utils.data import DataLoader
from torch.optim import LBFGS
        
    #from neuralNetworks.callbacks import LogMetricsData, LogMetrics
    #from neuralNetworks import callbacks
    #from quantumModels.QPINNBase import QPinnBase
    #from quantumModels.QuantumModels import QDNN, ProductFeatureMap

## Evolution Enhanced Feature Maps

Starting from paper: https://arxiv.org/abs/2011.10395

After the set of single qubit rotations (dataencoding), we consider another unitary exp(-iH$\tau$) which acts for time $\tau$ and is generated by the Hamiltonian $H$.
$H$ is chosen to be a complex many body Hamiltonian, to ensure that exponentially many amplitudes are generated.
(Additionally, we plan an evolution layer that is parameter dependent_ $\tau\rightarrow\tau(x)$.)

#### Differential Equations with highly nontrivial dynamics

$$\frac{\partial^2 u}{\partial x^2}=1 ~~~\text{with b.c.}~~~ u(x=-1)=u(x=1)=0 \\
\rightarrow u(x)=\frac{1}{2}(x^2-1)$$

For now: we chose a nearest neighbor Ising Hamiltonian $H=-J\sum_jZ_jZ_{j+1}+h\sum_jX_j$ with a fixed evolution time $\tau=2$ and J and h being drawn uniformly from (0,1].

We use AdamOptimizer with lr=0.01 & 200 operations. Set $u_0=0.75$ and train with 100 points. Cost choice of total magnetization in the $z-$direction ($\hat{C}\sum_jZ_j$) and a Hardware-Efficient Ansatz of depth=6 is used. 

#### Defining the Hamiltonian Time evolution:

In [ ]:
qml.ApproxTimeEvolution(H, tau, n) # I don't need to specify the wires as this is alrady done in H

#### Defining the Hamiltonian

In [4]:
n_qubits = 6

J = np.random.rand(1)
h = np.random.rand(1) 

obs1 = [qml.PauliZ(j)@qml.PauliZ(j+1) for j in range(n_qubits-1)]
obs2 = [qml.PauliX(j) for j in range(n_qubits)]

coeffs1 = [J for i in range(n_qubits-1)]
coeffs2 = [h for i in range(n_qubits)]

obs = obs1 + obs2
coeffs = coeffs1 + coeffs2

H = qml.Hamiltonian(coeffs, obs)

print(H)

  ([0.3481828]) [X0]
+ ([0.3481828]) [X1]
+ ([0.3481828]) [X2]
+ ([0.3481828]) [X3]
+ ([0.3481828]) [X4]
+ ([0.3481828]) [X5]
+ ([0.41191526]) [Z0 Z1]
+ ([0.41191526]) [Z1 Z2]
+ ([0.41191526]) [Z2 Z3]
+ ([0.41191526]) [Z3 Z4]
+ ([0.41191526]) [Z4 Z5]


#### Defining the Tower Chebyshev feature maps

In [13]:
n_qubits = 6
depth = 1
layers = 1

dev = qml.device("default.qubit", wires = n_qubits)

# parameters #
tau = 2  # evolution time
n = 10 # order of trotterization 

# Hamiltonian
J = np.random.rand(1)
h = np.random.rand(1) 

obs1 = [qml.PauliZ(j)@qml.PauliZ(j+1) for j in range(n_qubits-1)]
obs2 = [qml.PauliX(j) for j in range(n_qubits)]
coeffs1 = [J for i in range(n_qubits-1)]
coeffs2 = [h for i in range(n_qubits)]
obs = obs1 + obs2
coeffs = coeffs1 + coeffs2

H = qml.Hamiltonian(coeffs, obs)

# end parameters#


# Define possible feature maps

# Tower Chebyshev feature map
def TChebyshev(x):
    for layer in range(layers):
        for i in range(n_qubits):
            qml.RY(2*i*np.arccos(x), wires=i)

# Evolution ehanced feature map
def EvolutionEnhanced(x, H, tau = 2, n = 10):
    TChebyshev(x, H, tau, n)
    qml.ApproximateTimeEvolution(H, tau, n)


# Define the Hardware-Efficient Ansatz
def HardwareEffAnsatz(params):
    for _ in range(depth):
        for l in range(n_qubits):
            qml.RZ(params[l,0], wires=l)
            qml.RX(params[l,1], wires=l) 
            qml.RZ(params[l,2], wires=l)
        for l in range(0, n_qubits-1, 2):
            qml.CNOT(wires=[l,l+1])
        for l in range(1, n_qubits-1, 2):
            qml.CNOT(wires=[l,l+1])
        

@qml.qnode(dev)
def VQA_TC(x, params):
    TChebyshev(x)
    HardwareEffAnsatz(params)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)] # Use total magnetization as cost function
    

In [14]:
weights = np.random.randn(n_qubits, 3, requires_grad=True)
output=VQA_TC(np.pi/4, weights)
print(output)

ValueError: all input arrays must have the same shape

cost function: differential equation!

In [ ]:
def output(x, params,H):
    mz = VQA_TC(x, params)
    return np.sum(mz)

def residual(self, controlPoints, params, H):
        """ Calculate the residium of the control points"""
        x = controlPoints
        u = output(x, params, H)

        u_x = torch.autograd.grad(
            u, x,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True,
            allow_unused=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True,
            allow_unused=True
        )[0]
        f = u_xx - 1
        return f



def cost(params, x, H):
    return residual(x, params, H)**2



In [ ]:
opt = qml.AdamOptimizer(0.1)

weights = np.random.randn(n_qubits, 3, depth, requies_grad=True)
controlPoints = np.random.randn(100, requires_grad=False)

weights = opt.step(cost, weights, x, H)
weights = np.clip(-np.pi, np.pi)

In [109]:
def collocationPoints(xBounds, numSamples=100):
    """Creates the data-points to train and evaluate the quantum network"""
    seed = 42
    generator = np.random.default_rng(seed)
    pointsInside = np.expand_dims(generator.uniform(xBounds[0], xBounds[1], numSamples), 1)
    pointsBoundary = np.expand_dims(np.take(xBounds, np.random.randint(0, 2, numSamples)), 1)

    return (pointsInside, pointsBoundary)

In [110]:
# Problem related parameters
xBounds = (-1., 1.)
boundaryValues = (0., 0.)
numTrainPoints = 300
pointsInside, pointsBdr = collocationPoints(xBounds, numTrainPoints)
# Create dataset with collocation points
dataset = CreateTensorDataset(pointsInside, pointsBdr, backend="torch")

### Consider a rapidly oscillating non-persiodic solution.
Initial Value problem:
$$\frac{du}{dx}-4u+6u^2-sin(50x)-ucos(25x)+1/2 = 0, ~~~~~~ u(0)=u_0$$